# Dynamic error correction
This sample demonstrates how to create a 3 qubit repetition code that can be used to detect and correct bit flip errors. It leverages integrated hybrid computing features to count the number of times error correction was performed while the state of a logical qubit register is coherent.

Find more about integrated quantum computing at https://aka.ms/AQ/Hybrid/Docs

In this program you will learn to:
1. Encode a logical qubit using a 3 qubit repetition code.
2. Perform error correction when a bit flip error occurs.
3. Use classical compute to count the number of times error correction was performed.
## Connect to the Azure Quantum workspace
First, find the resource ID of your Azure Quantum workspace. You can copy/paste this from the top-right corner of your Quantum Workspace page in Azure Portal.

Next, you can run `%azure.connect` to connect to the workspace and display the list of provisioned targets that support execution of Q# programs. If you are prompted to login, be sure to use the same account you used to create your Azure Quantum workspace.

In [ ]:
%azure.connect "/subscriptions/subscriptionId/resourceGroups/resourceGroupName/providers/Microsoft.Quantum/Workspaces/workspaceName" location="westus"

## Select a target and opt-in to use integrated hybrid computing
First, we select the target the program is intended to be run on using the `%azure.target` command. This information is used by Q# compiler to check the program can be run on the specified target taking into consideration its capabilities.

Since integrated hybrid computing is an opt-in feature, we run the `%azure.target-capability` command to opt into it.


In [ ]:
%azure.target quantinuum.sim.h1-1e
%azure.target-capability AdaptiveExecution

## Open the required Q# namespaces
First, open the Q# namespaces that provide the necessary operations to implement this sample.


In [ ]:
open Microsoft.Quantum.Intrinsic;
open Microsoft.Quantum.Math;
open Microsoft.Quantum.Measurement;

## Encode a logical qubit using a repetition code
The following operation is used to encode a 3 qubit register into a single logical qubit using an error correcting repetition code.

In [ ]:
operation Encode(register : Qubit[]) : Unit is Adj {
    CNOT(register[0], register[1]);
    CNOT(register[0], register[2]);
}

## Measure a bit flip error syndromeThe following operation measures a bit flip error syndrome by checking the parities between qubits 0 and 1, and between qubits 1 and 2.

This operation uses a 2 qubit auxiliary register and leverages integrated hybrid computing capabilities: mid-program qubit measurement and qubit re-use. For an error correction program, these features reduce the number of qubits needed because otherwise we would have to use 2 additional qubits every time we wanted to measure the error syndrome.

In [ ]:
operation MeasureSyndrome(logicalRegister : Qubit[], auxiliaryRegister : Qubit[]) : (Result, Result)
{
    ResetAll(auxiliaryRegister);
    CNOT(logicalRegister[0], auxiliaryRegister[0]);
    CNOT(logicalRegister[1], auxiliaryRegister[0]);
    CNOT(logicalRegister[1], auxiliaryRegister[1]);
    CNOT(logicalRegister[2], auxiliaryRegister[1]);
    let parity01 = MResetZ(auxiliaryRegister[0]);
    let parity12 = MResetZ(auxiliaryRegister[1]);
    return (parity01, parity12);
}


## Use an error syndrome to correct errors
The following operation applies a corrective operation to a specific qubit in the register depending on the provided error syndrome and returns a boolean that represents whether any a correction was performed on any of the qubits
.
This operation uses another integrated hybrid computing capability, branching based on qubit measurements. As we can observe, a different quantum operation is performed depending on the measurement results.

In [ ]:
operation CorrectError(register : Qubit[], parity01 : Result, parity12 : Result) : Bool {
    if (parity01 == One and parity12 == Zero) {
         X(register[0]); 
    }
    elif (parity01 == One and parity12 == One) {
        X(register[1]);
    }
    elif (parity01 == Zero and parity12 == One) {
        X(register[2]);
    }

    return parity01 == One or parity12 == One;
}

## Implement an operation that could introduce errors
In order to test our error correction code, we can apply an operation to our logical qubit that is equivalent to applying an identity operation.
To achieve this, we can apply a few rotations about the x-axis such that after applying them the logical qubit is in the same state it was intially in.
Since the `Rx` operation has a $4\pi$ period, applying a $\frac{\pi}{2}$ rotation 8 times should return the qubit to its original state.
Note that since the register represents a logical qubit, rotations are applied to each qubit in the reguster.

The errors introduced by applying these rotations is what our program will attempt to correct.

In [ ]:
operation ApplyRotationalIdentity(register : Qubit[]) : Unit is Adj
{
    let theta = PI() * 0.5;
    for i in 1 .. 8 {
        for qubit in register
        {
            Rx(theta, qubit);
        }
    }
}


## Create a program to test our error correcting code
Now that we have implemented all the building blocks, we will integrate them all into a single program that we can submit to a quantum device.

Our program will do the following:
1. Initialize a qubit into a superposition state $|-〉$.
2. Encode the qubit into a logical qubit using a 3 qubit repetition code.
3. Apply several quantum operations to the encoded logical qubit performing error correction between each application..
4. Decode the logical qubit and measure it in the computational basis.
5. As output, return a boolean representing whether the qubit measurement was the expected one and an integer representing the number of times error correction was performed.

This program uses two more integrated hybrid computing features: classical compute during coherence time and additional output data types (boolean and iteger).


In [ ]:
operation DynamicBitFlipCode() : (Bool, Int) {
    // Use two qubit registers, one for representing a logical qubit and another one as auxiliary to detect the
    // error syndrome.
    use logicalRegister = Qubit[3];
    use auxiliaryRegister = Qubit[2];

    // Initialize the first qubit in the register to a |-〉 state.
    H(logicalRegister[0]);
    Z(logicalRegister[0]);

    // Apply several quantum operations to the encoded logical qubit performing error correction between each application.
    mutable corrections = 0;
    within {
        // The 3 qubit register is used to represent a single logical qubit using an error correcting repetition code.
        Encode(logicalRegister);
    }
    apply {
        let iterations = 5;
        for _ in 1 .. iterations {
            // Apply a sequence of rotations to the logical register that effectively perform an identity operation.
            ApplyRotationalIdentity(logicalRegister);

            // Measure the error syndrome, perform error correction based on it if needed, and increase the 
            // corrections counter if a correction was made.
            let (parity01, parity12) = MeasureSyndrome(logicalRegister, auxiliaryRegister);
            let correctedError = CorrectError(logicalRegister, parity01, parity12);
            if (correctedError) {
                set corrections += 1;
            }
        }
    }

    // Transform the qubit to the |1〉 state and measure it in the computational basis.
    H(logicalRegister[0]);
    let result = MResetZ(logicalRegister[0]) == One;
    ResetAll(logicalRegister);

    // The output of the program is a boolean-integer tuple where the boolean represents whether the qubit
    // measurement result was the expected one and the integer represents the number of times error correction was
    // performed.
    return (result, corrections);
}


## Submit the dynamic bit flip error correction program
### ❕ This program will use approximately 11.31 units (HQCs or eHQCs) using 50 shots, as configured below.
If the job has not been completed after the client times out (300 seconds), you can query the status using `%azure.status` followed by querying the output using `%azure.output`.<br/>

If the kernel was restarted after the program had been submitted, you will have to use `%azure.status <job_id>` to get the status of the job and `%azure.output <job_id>` to retrieve the results.<br/>
You can also check the status of the job under the **Job management** section on the left side menu of the portal.


In [ ]:
%azure.execute DynamicBitFlipCode shots=50 timeout=300 poll=10

In [ ]:
%azure.status

## Understanding the resultsEach item in the histogram is a boolean-integer tuple representing whether the qubit measurement was the expected one and the number of times error correction was performed. Associated to an item in the histogram is the frequency of that particular output.<br/>

For example:
- A `(true, 0)` output means that the qubit was measured in the expected state and no error correction was performed.
- A `(true, 2)` output means that the qubit was measured in the expected state and error correction was performed twice.
- A `(false, 1)` output means that the qubit was **not** measured in the expected state and error correction was performed once.


In [ ]:
%azure.output

## ↗ Learning more about Integrated Hybrid Computing
Now that you have run this sample, you can further learn and experiment with other samples available in the gallery.

To learn more, please refer to our [online documentation](https://aka.ms/AQ/Hybrid/Docs).